### EDA 코드 기록용_서연

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

#### 입학DB

In [ ]:
df = pd.read_csv("../../데이터/입학DB/01입학_클리닝.csv", converters={"ID" : str})

In [ ]:
## 각 컬럼별 유니크 값 카운트 및 비율 추출
for column in df.columns[1:]:
    print(f"[{column} value counts]")
    print(df[column].value_counts())
    print("-"*5,"개수 / 비율", "-"*5)
    print(round(df[column].value_counts(normalize=True)*100, 2))
    print("="*20)
    print()

In [ ]:
## 한 변수의 value별 취업률(1, 0) count해서 시각화

# 세부입학전형에 따른 취업률
# 0, 1 중 하나의 값만 있을 경우 KeyError 발생 -> pass
plt.figure(figsize=(25,25))

for i, count in enumerate(set(df_0["세부입학전형"])) :
    # 행, 열
    plt.subplot(5, 5, i+1)

    try: 
        sns.countplot(data=df_0[df_0["세부입학전형"]==count], x="취업구분_분석용")
    # value 값이 하나밖에 없어 오류가 날 경우 오류 알리고 pass
    except KeyError:
        print("KeyError 발생", count)
        pass
    finally:
        plt.title(f"세부입학전형 {count} - 취업 여부")

plt.show()

#### 학사DB (취경팀)

In [ ]:
# 학과별 최종취업구분 pie 시각화

for dept in df01["학과소분류명"].unique() :
    plt.figure(figsize=(6, 6))
    dept_count = df01[df01["학과소분류명"]==dept]["최종취업구분"].value_counts()
    plt.pie(dept_count.values, 
            labels=dept_count.index,
            colors=[]
            textprops={"fontsize":12})
    plt.title(f"{dept} 최종취업구분 비율")

In [ ]:
# 기업 연계용 프로그램 추출 (2개 이상의 값만 추출할 때)
df03_sub = df03[df03["프로그램명"].isin(program_list)]

In [ ]:
## 시트가 여러개인 엑셀 파일 import
# 4. 상담실적(2015~2020)
df04_2015 = pd.read_excel("../../데이터/취경팀/4. 상담실적_2015-2020년.xlsx", sheet_name="2015학년도(7753건)")

#### 학사DB

In [ ]:
# 컬럼별 유니크값 plot 추출
for column in df01.columns[1:]:
    plt.figure(figsize=(10, len(df01[column].unique())*0.5))
    sns.countplot(data=df01, y=column, order=df01[column].value_counts().index)

    plt.title(f"{column}")
    plt.show

In [ ]:
# 취득학위별 취업률
df129_0.groupby('취득학위')['취업구분_분석용'].value_counts(normalize=True).unstack().sort_values(by=1.0, ascending=False)

#### 학생DB

In [ ]:
# 한 학생이 몇 학기동안 동아리 활동을 했는지 -> 학번, 동아리 횟수

df_club_ = df_club.groupby("ID")["학기"].count().reset_index(name="동아리 횟수")

In [ ]:
from functools import reduce

# 여러 데이터프레임 한 번에 합치기
df_merge = reduce(lambda x, y: pd.merge(x, y, how='outer', on="ID"), [df0, df_club_, df_ss_, df_sc_])
df_merge.reset_index(drop=True, inplace=True) # 인덱스 초기화

----

#### 전처리 후 데이터 EDA

In [ ]:
## 상관관계
# heatmap 으로 시각화

def corr_heatmap(df, cols, title) :
    corr = df[cols].corr(method = 'pearson')

    fig = plt.figure(figsize = (18, 15))
    ax = fig.gca()

    # sns.set(font_scale = 1.5)  # heatmap 안의 font-size 설정
    heatmap = sns.heatmap(corr.values, annot = True, fmt='.2f', annot_kws={'size':15},
                        yticklabels = cols, xticklabels = cols, ax=ax, cmap = "RdYlBu")
    plt.tight_layout() ## 여백을 조정
    plt.title(title)
    plt.show()

corr_heatmap(admission_df, admission_df.columns[4:], "admission")

---

#### 최종 전처리 후 박스 플롯 출력

In [ ]:
# box plot - 성적 관련 컬럼

plt.figure(figsize=(15, 8))
plt.boxplot([ana_df["바종설_성적"], ana_df["1학년_취득성적"], ana_df["2학년_취득성적"], ana_df["3학년_취득성적"], ana_df["4학년_취득성적"], ana_df["전공_취득성적"], ana_df["교양_취득성적"], ana_df["전체성적"]], 
            labels = ["바종설_성적", "1학년_취득성적", "2학년_취득성적", "3학년_취득성적", "4학년_취득성적", "전공_취득성적", "교양_취득성적", "전체성적"])

# plt.savefig(f"2020/1.환경/plot/col_boxplot_{c}.png")
plt.title("성적 box plot")
plt.show()

In [ ]:
# box plot - 취경팀 비교과 관련 컬럼
# '진로탐색및설계_참여횟수', '직무탐색_참여횟수',
#             '역량강화_참여횟수', '취업실전_참여횟수',

plt.figure(figsize=(15, 8))
plt.boxplot([ana_df["진로탐색및설계_참여횟수"][ana_df["진로탐색및설계_참여횟수"] != 0],
             ana_df["직무탐색_참여횟수"][ana_df["직무탐색_참여횟수"] != 0], 
             ana_df["역량강화_참여횟수"][ana_df["역량강화_참여횟수"] != 0], 
             ana_df["취업실전_참여횟수"][ana_df["취업실전_참여횟수"] != 0]],
             labels = ["진로탐색및설계_참여횟수", "직무탐색_참여횟수", "역량강화_참여횟수", "취업실전_참여횟수"])

plt.title("비교과 box plot")
plt.show()

In [ ]:
# box plot - 취경팀 비교과 관련 컬럼
#교내장학금_누적금액', '교외장학금_누적금액', '국가장학금_누적금액'

plt.figure(figsize=(15, 8))
plt.boxplot([ana_df["교내장학금_누적금액"][ana_df["교내장학금_누적금액"] != 0],
             ana_df["교외장학금_누적금액"][ana_df["교외장학금_누적금액"] != 0], 
             ana_df["국가장학금_누적금액"][ana_df["국가장학금_누적금액"] != 0]],
             labels = ["교내장학금_누적금액", "교외장학금_누적금액", "국가장학금_누적금액"])

plt.title("장학금(0 제외) box plot")
plt.show()

In [ ]:
# box plot - 취경팀 비교과 관련 컬럼
#교내장학금_누적금액', '교외장학금_누적금액', '국가장학금_누적금액'

plt.figure(figsize=(15, 8))
plt.boxplot([ana_df["교내장학금_누적금액"],
             ana_df["교외장학금_누적금액"], 
             ana_df["국가장학금_누적금액"]],
             labels = ["교내장학금_누적금액", "교외장학금_누적금액", "국가장학금_누적금액"])

plt.title("장학금(0 포함) box plot")
plt.show()

In [ ]:
# box plot - 취경팀 상담 관련 컬럼 (0 제외)
# '진로상담_참여횟수', '취업상담_참여횟수'

plt.figure(figsize=(15, 8))
plt.boxplot([ana_df["진로상담_참여횟수"][ana_df["진로상담_참여횟수"] != 0],
             ana_df["취업상담_참여횟수"][ana_df["취업상담_참여횟수"] != 0]],
             labels = ["진로상담_참여횟수", "취업상담_참여횟수"])

plt.title("상담(0 제외) box plot")
plt.show()

In [ ]:
# box plot - 취경팀 상담 관련 컬럼
# '진로상담_참여횟수', '취업상담_참여횟수'

plt.figure(figsize=(15, 8))
plt.boxplot([ana_df["진로상담_참여횟수"],
             ana_df["취업상담_참여횟수"]],
             labels = ["진로상담_참여횟수", "취업상담_참여횟수"])

plt.title("상담 box plot")
plt.show()

In [ ]:
# box plot - 교내 활동 + 총이수학기 컬럼
# '동아리참여학기', '소학회참여학기', '총이수학기'

plt.figure(figsize=(15, 8))
plt.boxplot([ana_df["동아리참여학기"],
             ana_df["소학회참여학기"], 
             ana_df["총이수학기"]],
             labels = ["동아리참여학기", "소학회참여학기", "총이수학기"])

plt.title("교내 활동 + 총이수학기(0 포함) box plot")
plt.show()

In [ ]:
# box plot - 교내 활동 + 총이수학기 컬럼
# '동아리참여학기', '소학회참여학기', '총이수학기'

plt.figure(figsize=(15, 8))
plt.boxplot([ana_df["동아리참여학기"][ana_df["동아리참여학기"] != 0],
             ana_df["소학회참여학기"][ana_df["소학회참여학기"] != 0], 
             ana_df["총이수학기"][ana_df["총이수학기"] != 0]],
             labels = ["동아리참여학기", "소학회참여학기", "총이수학기"])

plt.title("교내 활동 + 총이수학기(0 제외) box plot")
plt.show()